In [ ]:
#| default_exp background_task

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


2023-01-04 12:21:44.187050: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[INFO] numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO] numexpr.utils: NumExpr defaulting to 8 threads.
[WARNING] airt.testing.activate_by_import: Failed to set gpu memory limit for tf; This could happen because of no gpu availability


In [ ]:
#| export

import os
import yaml
from pathlib import Path
from time import sleep
from typing import *

import shlex
import subprocess  # nosec B404
from subprocess import Popen  # nosec B404

import asyncio


import airt_service
import airt_service.sanitizer
from airt.logger import get_logger

In [ ]:
import nest_asyncio
import uvicorn
from fastapi.testclient import TestClient
from fastapi import BackgroundTasks, FastAPI, status, Response

from fastapi.openapi.docs import get_swagger_ui_html, get_redoc_html
from fastapi.openapi.utils import get_openapi
from fastapi.responses import FileResponse, RedirectResponse
from fastapi.staticfiles import StaticFiles

In [ ]:
#| exporti

logger = get_logger(__name__)

In [ ]:
logger = get_logger(__name__, level=10)

In [ ]:
# | export


async def execute_cli(
    command: str,
    timeout: int = 0,
    sleep_step: int = 1,
    on_timeout: Optional[Callable[[], None]] = None,
    on_success: Optional[Callable[[], None]] = None,
    on_error: Optional[Callable[[], None]] = None,
):
    """Execute CLI command

    Args:
        command: CLI command to be started in another process
        timeout: The maximum time allowed in seconds for the command to complete. If greater than 0,
                then the command will be killed after the timeout
        sleep_step: The time interval in seconds to check the completion status of the command
        on_timeout: Callback to be called in case the command gets killed due to a timeout
        on_success: Callback to be called if the command execution is successful, i.e., return code is 0
        on_error: Callback to be called if the command execution failed, i.e., return code is not 0

    Example:
        The following code executes a CLI command:
        ```python
        @app.post("/call_cli/{command}",  status_code=201)
        async def call_cli(command: str, background_tasks: BackgroundTasks, response: Response):
            background_tasks.add_task(
                execute_cli,
                command,
                timeout=3,
                on_timeout=lambda: logger.warning(f"Callback: background task timeouted for command: {command}")
            )
            response.code = 201
            return {"message": f"call_cli() returning after backgound task started for command: {command}"}
        ```
    """
    logger.info(f"Background task starting for command: '{command}'")
    cmd = shlex.split(command)
    logger.info(f"Background task command broken into: {cmd}")

    try:
        curr_env = os.environ.copy()
        curr_env["PATH"] = f"/home/{curr_env['USER']}/.local/bin:" + curr_env["PATH"]
        # start process
        # nosemgrep: python.lang.security.audit.dangerous-subprocess-use.dangerous-subprocess-use
        proc = Popen(  # nosec B603
            cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, env=curr_env
        )
    except Exception as e:
        logger.info(
            f"Background task thrown exception for command: '{command}' with exception {str(e)}"
        )
    logger.info(f"Background task started for command: '{command}'")

    i = 0
    while proc.poll() is None:
        if 0 < timeout <= i:
            logger.info(
                f"Background task timeouted after {i:,d} seconds for command: '{command}'"
            )
            logger.info(f"Killing background task command: '{command}'")
            proc.kill()
            while proc.poll() is None:
                logger.info(
                    f"Waiting for the background task to be killed for command: '{command}'"
                )
                await asyncio.sleep(sleep_step)
            logger.info(f"Background task killed for command: '{command}'")
            if on_timeout is not None:
                on_timeout()
            break

        logger.debug(
            f"Background task running for {i:,d} seconds for command: '{command}'"
        )
        await asyncio.sleep(sleep_step)
        i = i + sleep_step

    logger.info(f"Command finished with return code {proc.returncode}")
    if proc.returncode == 0:
        if on_success is not None:
            on_success()
    else:
        if on_error is not None:
            on_error()

    if proc.stdout is not None:
        logger.info(
            f"Background task stdout for command: '{command}':\n{proc.stdout.read()}"
        )
    if proc.stderr is not None:
        logger.info(
            f"Background task stderr for command: '{command}':\n{proc.stderr.read()}"
        )

    logger.info(
        f"Background task finished for command: '{command}' with return code {proc.returncode}"
    )

In [ ]:
app = FastAPI(title="Run CLI command in background")


@app.post("/call_cli/{command}", status_code=201)
async def call_cli(command: str, background_tasks: BackgroundTasks, response: Response):
    background_tasks.add_task(
        execute_cli,
        command,
        timeout=3,
        on_timeout=lambda: logger.warning(
            f"Callback: background task timeouted for command: {command}"
        ),
        on_success=lambda: logger.warning(
            f"Callback: background task successful for command: {command}"
        ),
        on_error=lambda: logger.warning(
            f"Callback: background task failed for command: {command}"
        ),
    )
    response.code = 201
    return {
        "message": f"call_cli() returning after backgound task started for command: {command}"
    }


client = TestClient(app)

for i in [2, 5]:
    command = f"sleep {i}"

    display("*" * 120)
    display(f"Test for command: {command}")
    display()

    response = client.post(f"/call_cli/{command}")

    assert response.status_code == 201

    actual = response.json()
    expected = {
        "message": f"call_cli() returning after backgound task started for command: sleep {i}"
    }
    assert actual == expected

    display(actual)

'************************************************************************************************************************'

'Test for command: sleep 2'

[DEBUG] asyncio: Using selector: EpollSelector
[INFO] __main__: Background task starting for command: 'sleep 2'
[INFO] __main__: Background task command broken into: ['sleep', '2']
[INFO] __main__: Background task started for command: 'sleep 2'
[DEBUG] __main__: Background task running for 0 seconds for command: 'sleep 2'
[DEBUG] __main__: Background task running for 1 seconds for command: 'sleep 2'
[INFO] __main__: Command finished with return code 0
[WARNING] __main__: Callback: background task successful for command: sleep 2
[INFO] __main__: Background task stdout for command: 'sleep 2':

[INFO] __main__: Background task stderr for command: 'sleep 2':

[INFO] __main__: Background task finished for command: 'sleep 2' with return code 0
[DEBUG] httpx._client: HTTP Request: POST http://testserver/call_cli/sleep%202 "HTTP/1.1 201 Created"


{'message': 'call_cli() returning after backgound task started for command: sleep 2'}

'************************************************************************************************************************'

'Test for command: sleep 5'

[DEBUG] asyncio: Using selector: EpollSelector
[INFO] __main__: Background task starting for command: 'sleep 5'
[INFO] __main__: Background task command broken into: ['sleep', '5']
[INFO] __main__: Background task started for command: 'sleep 5'
[DEBUG] __main__: Background task running for 0 seconds for command: 'sleep 5'
[DEBUG] __main__: Background task running for 1 seconds for command: 'sleep 5'
[DEBUG] __main__: Background task running for 2 seconds for command: 'sleep 5'
[INFO] __main__: Background task timeouted after 3 seconds for command: 'sleep 5'
[INFO] __main__: Killing background task command: 'sleep 5'
[INFO] __main__: Waiting for the background task to be killed for command: 'sleep 5'
[INFO] __main__: Background task killed for command: 'sleep 5'
[WARNING] __main__: Callback: background task timeouted for command: sleep 5
[INFO] __main__: Command finished with return code -9
[WARNING] __main__: Callback: background task failed for command: sleep 5
[INFO] __main__: Backgr

{'message': 'call_cli() returning after backgound task started for command: sleep 5'}

In [ ]:
# Test command where it will return non zero code
command = f"grep -q search_this file_doesnt_exist.txt"

display("*" * 120)
display(f"Test for command: {command}")
display()

response = client.post(f"/call_cli/{command}")

assert response.status_code == 201

actual = response.json()
expected = {
    "message": f"call_cli() returning after backgound task started for command: {command}"
}
assert actual == expected

display(actual)

'************************************************************************************************************************'

'Test for command: grep -q search_this file_doesnt_exist.txt'

[DEBUG] asyncio: Using selector: EpollSelector
[INFO] __main__: Background task starting for command: 'grep -q search_this file_doesnt_exist.txt'
[INFO] __main__: Background task command broken into: ['grep', '-q', 'search_this', 'file_doesnt_exist.txt']
[INFO] __main__: Background task started for command: 'grep -q search_this file_doesnt_exist.txt'
[INFO] __main__: Command finished with return code 2
[WARNING] __main__: Callback: background task failed for command: grep -q search_this file_doesnt_exist.txt
[INFO] __main__: Background task stdout for command: 'grep -q search_this file_doesnt_exist.txt':

[INFO] __main__: Background task stderr for command: 'grep -q search_this file_doesnt_exist.txt':
grep: file_doesnt_exist.txt: No such file or directory

[INFO] __main__: Background task finished for command: 'grep -q search_this file_doesnt_exist.txt' with return code 2
[DEBUG] httpx._client: HTTP Request: POST http://testserver/call_cli/grep%20-q%20search_this%20file_doesnt_exist.tx

{'message': 'call_cli() returning after backgound task started for command: grep -q search_this file_doesnt_exist.txt'}

In [ ]:
# patching async.run so we can run FastAPI within notebook (Jupyter started its own processing loop already)

nest_asyncio.apply()

In [ ]:
# | eval: false

# please open swagger and make the call

uvicorn.run(app, host="0.0.0.0", port=6006)

INFO:     Started server process [4427]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:6006 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [4427]
